In [96]:
import re


inputfile =  "D:\\phd-data\\NER-predictions-comparison-NL-EN-DE-multiple-methods\\Dutch\\GATE\\xml\\fold5.xml"
outputfile = "D:\\phd-data\\NER-predictions-comparison-NL-EN-DE-multiple-methods\\Dutch\\GATE\\bio\\fold5.txt"

with open (inputfile, "r", encoding="utf-8") as f:
    text = f.read()
    cleaned_text = re.sub(r"<([a-zA-Z0-9]+)([^>]+)>", r"<\1>", text) # take out attributes of xml tags
    cleaned_text = cleaned_text.replace('. \n',' .\n ') # tokenise points back
    cleaned_text = cleaned_text.replace('   ','  & ') # put ampersands back in
#     cleaned_text = cleaned_text.replace('( ed. )','( ed . )') # put space back in
#     cleaned_text = cleaned_text.replace(' sp.',' sp .') # put space back in
#     cleaned_text = cleaned_text.replace(' cf.',' cf .') # put space back in
#     cleaned_text = cleaned_text.replace(' comm.',' comm .') # put space back in
    cleaned_text = cleaned_text.replace('</ART>.','.</ART>').replace('</PER>.','.</PER>') # move points into tags    
    lines = cleaned_text.split('\n')
    lines = lines[2:] # take out first two lines, as they are xml and whitespace
    output = ""
    current_tag = False
    for line in lines:
        
        tokens = line.split()
        
        
        for token in tokens:
            
            if "<ART>" in token or "<PER>" in token: # start of tag
                entity = re.search(r"<(ART|PER)>",token)[1]
                tag = re.search(r"<(ART|PER)>",token)[0]
                                
                if "</ART>" in token or "</PER>" in token: # end of tag in same token
                    end_tag = re.search(r"</(ART|PER)>",token)[0]
                    output += f"{token.replace(tag,'').replace(end_tag,'')} B-{entity}\n"
                    current_tag = False
                else:
                    output += f"{token.replace(tag,'')} B-{entity}\n"
                    current_tag = entity
                    
            elif current_tag and ("</ART>" in token or "</PER>" in token): # in tag, found end
                output += f"{token.replace('</ART>','').replace('</PER>','')} I-{current_tag}\n"
                current_tag = False
                
            elif current_tag: # in tag, not found end yet
                output += f"{token} I-{current_tag}\n"
                
            else: # output O
                output += f"{token} O\n"
                
        #print(line)
        output += '\n' #extra line ending to show end of sentence
        
#print(output)
with open(outputfile, "w", encoding="utf-8") as file:
    file.write(output)

